In [ ]:
!pip install music21

In [ ]:
import music21

In [ ]:
#from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Fonction général qui appelle les autre fonction et entraine le model LSTM

In [ ]:
def train_network():
    
    notes = get_notes()

    n = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n)

    model = create_model(network_input, n)

    train(model, network_input, network_output)

In [ ]:
def get_notes():
    
   # Nous récupérons toutes les notes et les accords des fichiers midi dans le répertoire ./midi_songs.
    notes_list = []

    for f in glob.glob("/content/drive/MyDrive/midi_songs/*.mid"):
        midi = converter.parse(f)

        print("Parsing %s" % f)

        notes = None

        try: 
            temp = instrument.partitionByInstrument(midi)
            notes = temp.parts[0].recurse() 
        except: 
            notes = midi.flat.notes

        for element in notes:
            if isinstance(element, note.Note):
                notes_list.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes_list.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/drive/MyDrive/data/notes_list', 'wb') as filepath:
        pickle.dump(notes_list, filepath)

    return notes_list

In [ ]:
def prepare_sequences(notes_list, n):
    
    # La taille de la sequence peut être modifiée, plus elle est grande plus le model pourra apprendre à produire une mélodie variée cependant cela augment considerablement le temps de training
    sequence_length = 30

   

    # Nous récuperons les différentes hauteurs
    pitch= sorted(set(notes_list))
   

    # Nous créons un dictionnaire pour mapper chaque hauteur à un entier
    note_to_int = dict((note, integer) for integer, note in enumerate(pitch))

    network_input = []
    network_output = []

    # Création de la séquence d'entrée et de la sortie correspondante
    for i in range(0, len(notes_list) - sequence_length, 1):
        sequence_in = notes_list[i:i + sequence_length]
        sequence_out = notes_list[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # On utilise numpy pour mettre l'input dans un format compatible avec LSTM
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
 
    network_input = network_input / float(n)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
def create_model(network_input, n):
    
    # Création de la structure du réseau neuronal
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
def train(model, network_input, network_output):
    
    # Entrainement du réseau neuronal
    
    filepath = "/content/drive/MyDrive/data/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)

In [ ]:
train_network()

Parsing /content/drive/MyDrive/midi_songs/d-123trainwithme.mid
Parsing /content/drive/MyDrive/midi_songs/d-100purelove.mid
Parsing /content/drive/MyDrive/midi_songs/d-closetoyou.mid
Parsing /content/drive/MyDrive/midi_songs/d-dadip.mid
Parsing /content/drive/MyDrive/midi_songs/d-dancingwithanangel.mid
Parsing /content/drive/MyDrive/midi_songs/d-awayfromhome.mid
Parsing /content/drive/MyDrive/midi_songs/d-childrenofthenight.mid
Parsing /content/drive/MyDrive/midi_songs/celebrat_d.mid
Parsing /content/drive/MyDrive/midi_songs/d-anotherday.mid
Parsing /content/drive/MyDrive/midi_songs/d-badboy.mid
Parsing /content/drive/MyDrive/midi_songs/blkisblk_d.mid
Parsing /content/drive/MyDrive/midi_songs/d-becauseimlovingyou.mid
Parsing /content/drive/MyDrive/midi_songs/d-automaticlover.mid
Parsing /content/drive/MyDrive/midi_songs/d-bemylover.mid
Parsing /content/drive/MyDrive/midi_songs/d-comenrideit.mid
Parsing /content/drive/MyDrive/midi_songs/d-cocojamboo.mid
Parsing /content/drive/MyDrive/mid